# Download JPG file

In [ ]:
%mkdir first100

In [ ]:
!tar -xzvf first100.tar.gz -C /content/first100

In [ ]:
!unzip chest-imagenome-dataset-1.0.0.zip -d /content/

In [ ]:
%cd /content/chest-imagenome-dataset-1.0.0/silver_dataset/

In [ ]:
!unzip scene_graph.zip

In [ ]:
%cd /content

In [ ]:
import json

sample_json = '02aa804e-bde0afdd-112c0b34-7bc16630-4e384014_SceneGraph.json'
with open(sample_json, 'r') as f:
    data = json.load(f)
data

In [ ]:
import os
import json
import random
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
image_folder = '/content/first100/'
scene_graph_folder = '/content/chest-imagenome-dataset-1.0.0/silver_dataset/scene_graph/'

# Show the original image of Chest ImaGenome

In [ ]:
def normalize_bbox(bbox, img_width, img_height):
    """
    Normalize the bounding box
    """
    x1_norm = bbox['original_x1'] / img_width
    y1_norm = bbox['original_y1'] / img_height
    x2_norm = bbox['original_x2'] / img_width
    y2_norm = bbox['original_y2'] / img_height
    return x1_norm, y1_norm, x2_norm, y2_norm


In [ ]:
def adjust_text_position(bbox, img_width, img_height):
    """
    Adjust text position to avoid overlap and ensure it's inside the image.

    Args:
    - bbox (dict): Dictionary with normalized coordinates.
    - img_width (int): Image width.
    - img_height (int): Image height.

    Returns:
    - (x, y): Adjusted x, y coordinates for the text label.
    """
    corners = [
        (bbox['x1_norm'], bbox['y1_norm']),
        (bbox['x2_norm'], bbox['y1_norm']),
        (bbox['x2_norm'], bbox['y2_norm']),
        (bbox['x1_norm'], bbox['y2_norm'])
    ]
    selected_corner = random.choice(corners)
    x_text = selected_corner[0] * img_width
    y_text = selected_corner[1] * img_height

    x_offset = 5
    y_offset = 5

    x_text += x_offset
    y_text += y_offset

    if x_text > img_width - 100:
        x_text = img_width - 100
    if y_text > img_height - 50:
        y_text = img_height - 50
    if x_text < 0:
        x_text = 0
    if y_text < 0:
        y_text = 0

    return x_text, y_text

In [ ]:
%cd /content/

In [ ]:
anatomies = [
    "abdomen", "aortic arch", "cardiac silhouette", "carina", "cavoatrial junction",
    "descending aorta", "left apical zone", "left cardiac silhouette", "left cardiophrenic angle",
    "left clavicle", "left costophrenic angle", "left hemidiaphragm", "left hilar structures",
    "left lower lung zone", "left lung", "left mid lung zone", "left upper abdomen",
    "left upper lung zone", "mediastinum", "right apical zone", "right atrium",
    "right cardiac silhouette", "right cardiophrenic angle", "right clavicle",
    "right costophrenic angle", "right hemidiaphragm", "right hilar structures",
    "right lower lung zone", "right lung", "right mid lung zone", "right upper abdomen",
    "right upper lung zone", "spine", "svc", "trachea", "upper mediastinum"
]

def display_bboxes_for_identifier(unique_id, scene_graph_folder, image_folder, bbox_color='red'):
    """
    Display bounding boxes on the image with numbers representing each anatomy.

    Args:
    - unique_id (str): Unique identifier of the image.
    - scene_graph_folder (str): Folder where the metadata (JSON files) are stored.
    - image_folder (str): Folder where the images (JPEG files) are stored.
    - bbox_color (str): Color for the bounding boxes (default is 'red').
    """

    json_file = f"{unique_id}_SceneGraph.json"
    json_path = os.path.join(scene_graph_folder, json_file)

    if not os.path.exists(json_path):
        print(f"JSON file not found for unique identifier: {unique_id}")
        return

    image_path = None
    for root, dirs, files in os.walk(image_folder):
        for file in files:
            if file.endswith(".jpg") and file.replace(".jpg", "") == unique_id:
                image_path = os.path.join(root, file)
                break

    if image_path is None:
        print(f"No matching image found for unique identifier: {unique_id}")
        return

    img = Image.open(image_path).convert("L")
    img_width, img_height = img.size

    with open(json_path, 'r') as f:
        data = json.load(f)

    bbox_data = []
    for obj in data['objects']:
        x1_norm, y1_norm, x2_norm, y2_norm = normalize_bbox(obj, img_width, img_height)

        anatomy_name = obj['bbox_name']
        if anatomy_name in anatomies:
            anatomy_index = anatomies.index(anatomy_name)
        else:
            anatomy_index = "?"

        bbox_data.append({
            'anatomy_index': anatomy_index,
            'x1_norm': x1_norm,
            'y1_norm': y1_norm,
            'x2_norm': x2_norm,
            'y2_norm': y2_norm
        })


    num_boxes_per_image = 4
    fig, axs = plt.subplots(3, 3, figsize=(18, 18))
    fig.tight_layout()

    for i in range(0, len(bbox_data), num_boxes_per_image):
        row = (i // num_boxes_per_image) // 3
        col = (i // num_boxes_per_image) % 3
        ax = axs[row, col]
        ax.imshow(img, cmap='gray')
        ax.set_axis_off()
        for bbox in bbox_data[i:i + num_boxes_per_image]:
            rect = patches.Rectangle((bbox['x1_norm'] * img_width, bbox['y1_norm'] * img_height),
                                     (bbox['x2_norm'] - bbox['x1_norm']) * img_width,
                                     (bbox['y2_norm'] - bbox['y1_norm']) * img_height,
                                     linewidth=1, edgecolor=bbox_color, facecolor='none')
            ax.add_patch(rect)

            x_text = bbox['x1_norm'] * img_width
            y_text = (bbox['y1_norm'] * img_height) - 10

            if bbox['anatomy_index'] == 2:
                x_text += 150
                y_text -= 20

            ax.text(x_text, y_text, str(bbox['anatomy_index']), color='white', fontsize=12,
                    bbox=dict(facecolor=bbox_color, alpha=0.5))

        if (i // num_boxes_per_image) == 8:
            break

    pdf_filename = f"{unique_id}.pdf"
    plt.savefig(pdf_filename, format="pdf")
    plt.show()

    print(f"Figure saved as {pdf_filename}")


In [ ]:
%pwd

In [ ]:
unique_identifier = '02aa804e-bde0afdd-112c0b34-7bc16630-4e384014'
display_bboxes_for_identifier(unique_identifier, scene_graph_folder, image_folder, bbox_color='black')

In [ ]:
def display_bboxes_for_identifier(unique_id, scene_graph_folder, image_folder, bbox_color='red'):
    """
    According to the unique identifier, match the image and metadata, and paint bounding boxes on the image.

    Args:
    - unique_id (str): Unique identifier of the image.
    - scene_graph_folder (str): Folder where the metadata (JSON files) are stored.
    - image_folder (str): Folder where the images (JPEG files) are stored.
    - bbox_color (str): Color for the bounding boxes (default is 'red').
    """

    json_file = f"{unique_id}_SceneGraph.json"
    json_path = os.path.join(scene_graph_folder, json_file)

    if not os.path.exists(json_path):
        print(f"JSON file not found for unique identifier: {unique_id}")
        return

    image_path = None
    for root, dirs, files in os.walk(image_folder):
        for file in files:
            if file.endswith(".jpg") and file.replace(".jpg", "") == unique_id:
                image_path = os.path.join(root, file)
                break

    if image_path is None:
        print(f"No matching image found for unique identifier: {unique_id}")
        return

    img = Image.open(image_path).convert("L")
    img_width, img_height = img.size

    with open(json_path, 'r') as f:
        data = json.load(f)

    bbox_data = []
    for obj in data['objects']:
        x1_norm, y1_norm, x2_norm, y2_norm = normalize_bbox(obj, img_width, img_height)
        bbox_data.append({
            'bbox_name': obj['bbox_name'],
            'x1_norm': x1_norm,
            'y1_norm': y1_norm,
            'x2_norm': x2_norm,
            'y2_norm': y2_norm
        })

    num_boxes_per_image = 4
    fig, axs = plt.subplots(3, 3, figsize=(18, 18))
    fig.tight_layout()

    for i in range(0, len(bbox_data), num_boxes_per_image):
        row = (i // num_boxes_per_image) // 3
        col = (i // num_boxes_per_image) % 3
        ax = axs[row, col]
        ax.imshow(img, cmap='gray')
        ax.set_axis_off()
        for bbox in bbox_data[i:i + num_boxes_per_image]:
            print(bbox['bbox_name'])
            rect = patches.Rectangle((bbox['x1_norm'] * img_width, bbox['y1_norm'] * img_height),
                                     (bbox['x2_norm'] - bbox['x1_norm']) * img_width,
                                     (bbox['y2_norm'] - bbox['y1_norm']) * img_height,
                                     linewidth=1, edgecolor=bbox_color, facecolor='none')
            ax.add_patch(rect)
            x_text, y_text = adjust_text_position(bbox, img_width, img_height)

            ax.text(x_text, y_text, bbox['bbox_name'], color='white', fontsize=8,
                    bbox=dict(facecolor=bbox_color, alpha=0.5))

        if (i // num_boxes_per_image) == 8:
            break

    plt.show()

In [ ]:
unique_identifier = '02aa804e-bde0afdd-112c0b34-7bc16630-4e384014'
display_bboxes_for_identifier(unique_identifier, scene_graph_folder, image_folder, bbox_color='black')

In [ ]:
unique_identifier = '2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab'
display_bboxes_for_identifier(unique_identifier, scene_graph_folder, image_folder, bbox_color='black')

In [ ]:
unique_identifier = '6ad03ed1-97ee17ee-9cf8b320-f7011003-cd93b42d'
display_bboxes_for_identifier(unique_identifier, scene_graph_folder, image_folder, bbox_color='black')

In [ ]:
unique_identifier = 'd0b71acc-b5a62046-bbb5f6b8-7b173b85-65cdf738'
display_bboxes_for_identifier(unique_identifier, scene_graph_folder, image_folder, bbox_color='black')

In [ ]:
with open('/content/eval_all_batches_ep2_singleInstruction_formatted.json', 'r') as f:
    data = json.load(f)

In [ ]:
data[:6]

# Visualize the predictions and ground truth

In [ ]:
!unzip /content/llava_visualization.zip -d /content/

In [ ]:
result = [
    {
        'image_path': '/content/llava_visualization/5e2975f2-50307bfd-505b04f0-7d213948-6089450c.jpg',
        'prediction': [{'left lung':[0.56, 0.09, 0.9, 0.96]}],
        'ground_truth': [{'left lung': [0.56, 0.12, 0.87, 0.95]}]
    },
    {
        'image_path': '/content/llava_visualization/1cd8fc53-9c2a5c2a-cfb5af51-75828560-4df0989b.jpg',
        'prediction': [{'carina':[0.47, 0.33, 0.52, 0.38]}],
        'ground_truth': [{'carina': [0.48, 0.37, 0.5, 0.4]}]
    },
    {
        'image_path': '/content/llava_visualization/2d3cab73-18a0452c-fec484bd-19e19c3f-bff3d526.jpg',
        'prediction': [{'aortic arch':[0.5, 0.23, 0.58, 0.31], 'left clavicle':[0.54, 0.06, 0.93, 0.21]}],
        'ground_truth': [{'aortic arch':[0.5, 0.29, 0.56, 0.34], 'left clavicle':[0.53, 0.09, 0.86, 0.24]}]
    },
    {
        'image_path': '/content/llava_visualization/6e72579d-27568b2c-c937c698-178e017f-d96ba14e.jpg',
        'prediction': [{'cardiac silhouette':[0.5, 0.23, 0.58, 0.31], 'upper mediastinum':[0.54, 0.06, 0.93, 0.21]}],
        'ground_truth': [{'aortic arch':[0.37, 0.48, 0.75, 0.83], 'left clavicle':[0.39, 0.2, 0.58, 0.47]}]
    },
    {
        'image_path': '/content/llava_visualization/a7b649f8-a9792071-ebc05c65-17bef53e-738c48bb.jpg',
        'prediction': [{'left mid lung zone':[0.47, 0.25, 0.83, 0.4], 'left upper lung zone':[0.5, 0.04, 0.83, 0.25], 'right atrium':[0.25, 0.42, 0.42, 0.58]}],
        'ground_truth': [{'left mid lung zone': [0.46, 0.27, 0.79, 0.42], 'left upper lung zone': [0.47, 0.08, 0.78, 0.27], 'right atrium': [0.31, 0.43, 0.44, 0.6]}]
    },
    {
        'image_path': '/content/llava_visualization/1a9b3b82-402d9e19-445679da-a0accb94-ebc21d66.jpg',
        'prediction': [{'right apical zone': [0.18, 0.17, 0.44, 0.31], 'left hemidiaphragm': [0.5, 0.75, 0.87, 0.86], 'right hilar structures': [0.28, 0.38, 0.44, 0.61]}],
        'ground_truth': [{'right apical zone': [0.23, 0.25, 0.45, 0.38], 'left hemidiaphragm': [0.5, 0.72, 0.82, 0.82], 'right hilar structures': [0.31, 0.42, 0.44, 0.6]}]
    },
    {
        'image_path': '/content/llava_visualization/7cf9a8eb-4cff16e3-b01b5d1a-7c127cd6-607372be.jpg',
        'prediction': [{'right apical zone': [0.12, 0.08, 0.4, 0.25], 'left hemidiaphragm': [0.47, 0.58, 0.87, 0.69], 'cavoatrial junction': [0.29, 0.4, 0.46, 0.48], 'right clavicle': [0.0, 0.09, 0.38, 0.25], 'left upper abdomen': [0.47, 0.61, 0.87, 1.0]}],
        'ground_truth': [{'right apical zone': [0.17, 0.09, 0.41, 0.25], 'left hemidiaphragm': [0.46, 0.53, 0.81, 0.69], 'cavoatrial junction': [0.33, 0.43, 0.46, 0.5], 'right clavicle': [0.0, 0.12, 0.35, 0.25], 'left upper abdomen': [0.46, 0.55, 0.81, 1.0]}]
    },
    {
        'image_path': '/content/llava_visualization/4526ef7e-a5046703-0866561b-081b0533-53dae6d4.jpg',
        'prediction': [{'svc': [0.39, 0.26, 0.48, 0.5], 'right cardiophrenic angle': [0.3, 0.72, 0.38, 0.85], 'spine': [0.42, 0.0, 0.58, 1.0], 'left clavicle':[0.54, 0.14, 0.99, 0.28]}],
        'ground_truth': [{'svc': [0.39, 0.33, 0.47, 0.53], 'right cardiophrenic angle': [0.31, 0.69, 0.38, 0.8], 'spine': [0.42, 0.0, 0.54, 1.0], 'left clavicle': [0.53, 0.21, 0.88, 0.37]}]
    },
 ]

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches

def visualize_results(result, images_per_row=4):
    """
    Visualize predictions and ground truth for chest X-ray images.

    Parameters:
    result (list): List of dictionaries containing image paths, predictions, and ground truths.
    images_per_row (int): Number of images to plot per row.
    """
    num_images = len(result)
    rows = (num_images // images_per_row) + (num_images % images_per_row > 0)

    fig, axes = plt.subplots(rows, images_per_row, figsize=(20, 5 * rows))
    axes = axes.flatten()

    for idx, data in enumerate(result):
        try:

            img = Image.open(data['image_path']).convert('L')
            ax = axes[idx]
            ax.imshow(img, cmap='gray')
            ax.axis('off')
            ax.set_title(f"Sample 0{idx + 1}")

            for gt in data['ground_truth']:
                for label, bbox in gt.items():
                    x_min, y_min, x_max, y_max = bbox
                    rect = patches.Rectangle((x_min * img.width, y_min * img.height),
                                             (x_max - x_min) * img.width, (y_max - y_min) * img.height,
                                             linewidth=2, edgecolor='green', facecolor='none')
                    ax.add_patch(rect)

                    ax.text(x_min * img.width, y_max * img.height + 5, label, color='green', fontsize=10,
                            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=0.3))

            for pred in data['prediction']:
                for bbox in pred.values():
                    x_min, y_min, x_max, y_max = bbox
                    rect = patches.Rectangle((x_min * img.width, y_min * img.height),
                                             (x_max - x_min) * img.width, (y_max - y_min) * img.height,
                                             linewidth=2, edgecolor='red', facecolor='none')
                    ax.add_patch(rect)
        except Exception as e:
            print(f"Could not process image {idx + 1}: {e}")

    for i in range(len(result), len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.savefig('chest_imagenome_sample.pdf', format = 'pdf')
    plt.show()


In [ ]:
visualize_results(result)